<a href="https://colab.research.google.com/github/nausheen53/SMAI/blob/master/Q5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import ssl
import sklearn
import numpy as np
import spacy
import re
import gensim
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
import textblob
from textblob import TextBlob, Word
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

In [3]:

df = pd.read_csv('/content/drive/My Drive/Train.csv',header = 0)
print(len(df))

15663


In [0]:
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [5]:
train = np.array(train)
test = np.array(test)
labels_of_train=[]
for row in train:
    labels_of_train.append(row[2])
text_of_train = []
for row in train:
    text_of_train.append(row[1])
print(len(labels_of_train))    

labels_of_test=[]
for row in test:
    labels_of_test.append(row[2])
text_of_test = []
for row in test:
    text_of_test.append(row[1])

12530


In [0]:
import string
from collections import Counter
final_list_train=[]
for sent in text_of_train:
    # print("1")
    sent = sent.lower()
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    # print("sent1 ",sent)
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(sent)
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha()]
    #remove stopwords
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    sent=" ".join(words)
    # print("sentttt ",sent)
    final_list_train.append(sent)
    
    

In [0]:
import string
from collections import Counter
final_list_test=[]
for sent in text_of_test:
    # print("1")
    sent = sent.lower()
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    # print("senttttt ",sent)
#     tokens= sent.split()
    ###################################################
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(sent)
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha()]
    #remove stopwords
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    sent=" ".join(words)
    final_list_test.append(sent)

In [20]:
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [21]:
print(model.vector_size)

300


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
text = final_list_train
vectorizer = TfidfVectorizer()
vectorizer.fit(text)
vector = vectorizer.fit_transform(text)
vector = vector.todense()
vector1 = vectorizer.transform(final_list_test)
vector1 = vector1.todense()
words = vectorizer.get_feature_names()
# print(words)
vector_norm_train = normalize(vector)
print(vector_norm_train)
print(vector_norm_train.shape)

clf = svm.SVC(kernel='linear', C = 4.0)
print((vector_norm_train).shape)
print(len(labels_of_train))
clf.fit(vector,labels_of_train)
print(clf.predict(vector1))
predicted_label=clf.predict(vector1)    
accu_score = accuracy_score(labels_of_test, predicted_label,normalize=True)
print(accu_score)  

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(12530, 21283)
(12530, 21283)
12530
['EAP' 'MWS' 'MWS' ... 'EAP' 'HPL' 'MWS']
0.7867858282796042


In [9]:
f11 = f1_score(labels_of_test, predicted_label,average=None)
re = recall_score(labels_of_test, predicted_label,average='weighted')
cm = confusion_matrix(labels_of_test, predicted_label)
pr_sc = precision_score(labels_of_test, predicted_label,average='weighted')
clrp = classification_report(labels_of_test, predicted_label)
print("accuracy score ",accu_score)
print("********************")
print("f1 score ",f11)
print("********************")
print("recall score ",re)
print("********************")
print("confusion matrix ",cm)
print("********************")
print("precision score ",pr_sc)
print("********************")
print("classification report ",clrp)

accuracy score  0.7867858282796042
********************
f1 score  [0.79774011 0.76965979 0.78767877]
********************
recall score  0.7867858282796042
********************
confusion matrix  [[1059  114  118]
 [ 168  690   61]
 [ 137   70  716]]
********************
precision score  0.7871846886602186
********************
classification report                precision    recall  f1-score   support

         EAP       0.78      0.82      0.80      1291
         HPL       0.79      0.75      0.77       919
         MWS       0.80      0.78      0.79       923

    accuracy                           0.79      3133
   macro avg       0.79      0.78      0.79      3133
weighted avg       0.79      0.79      0.79      3133

